In [50]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

In [51]:
vest_ak_20 = gp.read_file("./raw-from-source/VEST/ak_2020/ak_2020.shp")

In [52]:
print(vest_ak_20.shape)

(441, 15)


In [53]:
vest_ak_20.head(2)

,DISTRICT,NAME,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREGJAN,G20PRECBLA,G20PREIPIE,G20PREOFUE,G20USSRSUL,G20USSDGRO,G20USSOHOW,G20HALRYOU,G20HALDGAL,geometry
0,11-075,11-075 Palmer City No. 2,685,397,46,12,7,0,0,656,421,71,686,441,"POLYGON ((-149.13365 61.61026, -149.13361 61.6..."
1,12-220,12-220 Butte,1808,689,62,16,4,3,0,1761,673,96,1780,751,"POLYGON ((-148.52103 61.62682, -148.52020 61.6..."


Election results from Alaska Division of Elections (http://www.elections.alaska.gov/results/20GENR/index.php)
Precinct shapefile from Alaska Division of Elections (http://www.elections.alaska.gov/Core/districtmaps.php)

Early, Absentee, and Questioned votes are only reported at the State House district level (since Alaska has portions of the state with no lower level of government, HDs serve a similar purpose as counties do in other states for the purpose of reporting votes). These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within an HD. Similarly, federal-only ballots that are reported at the statewide level were apportioned to precincts by candidate based on their share of the precinct-level vote.

G20PRERTRU - Donald J. Trump (Republican Party)
G20PREDBID - Joseph R. Biden (Democratic Party)
G20PRELJOR - Jo Jorgensen (Libertarian Party)
G20PREGJAN - James G. "Jesse Ventura" Janos (Green Party)
G20PRECBLA - Don Blankenship (Conservative Party)
G20PREIPIE - Brock Pierce (Independent)
G20PREOFUE - Roque "Rocky" De La Fuente (Alliance Party)

G20USSRSUL - Dan Sullivan (Republican Party)
G20USSDGRO - Al Gross (Democratic Party)
G20USSOHOW - John Wayne Howe (Alaskan Independence Party)

G20HALRYOU - Don Young (Republican Party)
G20HALDGAL - Alyse S. Galvin (Democratic Party)

In [54]:
ak_2020 = pd.read_csv("./raw-from-source/Election_Results/resultsbyprecinct.txt",header=None)


In [55]:
ak_2020.head(1)

,0,1,2,3,4,5,6,7,8
0,HD99 Fed Overseas Absentee,Race Statistics,NP,NP,Number of Precincts,NP,Total,0,NaN


In [56]:
ak_2020.columns = ["Precinct","Race","Blank_1","Blank_2","Choice","Party","Type","Votes","Extra"]
ak_2020 = ak_2020 [["Precinct","Race","Choice","Party","Votes"]]

In [57]:
print(ak_2020["Race"].unique())

['Race Statistics' 'U.S. President / Vice President""' 'U.S. Senator""'
 'U.S. Representative""' 'House District 1""' 'Supreme Court - Carney""'
 'Court of Appeals - Wollenberg""' 'Superior Court JD4 - Peters""'
 'District Court JD4 - Christian""' 'District Court JD4 - Montgomery""'
 'Ballot Measure No. 1 - 19OGTX""' 'Ballot Measure No. 2 - 19AKBE""'
 'House District 2""' 'Senate District B""' 'House District 3""'
 'House District 4""' 'House District 5""' 'House District 6""'
 'Superior Court JD3 - Crosby""' 'Superior Court JD3 - Guidi""'
 'Superior Court JD3 - Henderson""' 'Superior Court JD3 - Lamoureux""'
 'Superior Court JD3 - Miller""' 'Superior Court JD3 - Reigh""'
 'Superior Court JD3 - Wells""' 'Superior Court JD3 - Woodman""'
 'District Court JD3 - Dickson""' 'District Court JD3 - Franciosi""'
 'District Court JD3 - Hanley""' 'District Court JD3 - Logue""'
 'District Court JD3 - McCrea""' 'District Court JD3 - Wallace""'
 'District Court JD3 - Washington""' 'Senate District D

In [58]:
race_list = ['U.S. President / Vice President""', 'U.S. Senator""','U.S. Representative""']
ak_2020 = ak_2020[ak_2020["Race"].isin(race_list)]

In [59]:
ak_2020["Choice"].unique()

array(['Number of Precincts for Race', 'Number of Precincts Reporting',
       'Registered Voters', 'Times Counted',
       'Biden  Joseph R. Jr. / Harris  Kamala D.""',
       'Blankenship  Don / Mohr  William""',
       'De La Fuente  Rocque "Rocky" / Richardson  Darcy G.""',
       'Janos  James G. "Jesse Ventura" / McKinney  Cynthia""',
       'Jorgensen  Jo / Cohen  Jeremy "Spike"""',
       'Pierce  Brock / Ballard  Karla""',
       'Trump  Donald J. / Pence  Michael R.""', 'Gross  Al""',
       'Howe  John Wayne""', 'Sullivan  Dan""', 'Galvin  Alyse S.""',
       'Young  Don""'], dtype=object)

In [60]:
choice_list = ['Biden  Joseph R. Jr. / Harris  Kamala D.""',
       'Blankenship  Don / Mohr  William""',
       'De La Fuente  Rocque "Rocky" / Richardson  Darcy G.""',
       'Janos  James G. "Jesse Ventura" / McKinney  Cynthia""',
       'Jorgensen  Jo / Cohen  Jeremy "Spike"""',
       'Pierce  Brock / Ballard  Karla""',
       'Trump  Donald J. / Pence  Michael R.""', 'Gross  Al""',
       'Howe  John Wayne""', 'Sullivan  Dan""', 'Galvin  Alyse S.""',
       'Young  Don""']
ak_2020 = ak_2020[ak_2020["Choice"].isin(choice_list)]

In [64]:
ak_2020_pvt = pd.pivot_table(ak_2020,index=["Precinct"],values=["Votes"],columns=["Choice"],aggfunc=sum)
ak_2020_pvt.columns=ak_2020_pvt.columns.droplevel(0)
ak_2020_pvt.reset_index(inplace=True,drop=False)

In [65]:
ak_2020_pvt.head(1)

Choice,Precinct,"Biden Joseph R. Jr. / Harris Kamala D.""""","Blankenship Don / Mohr William""""","De La Fuente Rocque ""Rocky"" / Richardson Darcy G.""""","Galvin Alyse S.""""","Gross Al""""","Howe John Wayne""""","Janos James G. ""Jesse Ventura"" / McKinney Cynthia""""","Jorgensen Jo / Cohen Jeremy ""Spike""""""","Pierce Brock / Ballard Karla""""","Sullivan Dan""""","Trump Donald J. / Pence Michael R.""""","Young Don"""""
0,01-446 Aurora,239,3,0,293,246,46,2,43,3,465,466,452


In [66]:
print(ak_2020_pvt.columns)

Index(['Precinct', 'Biden  Joseph R. Jr. / Harris  Kamala D.""',
       'Blankenship  Don / Mohr  William""',
       'De La Fuente  Rocque "Rocky" / Richardson  Darcy G.""',
       'Galvin  Alyse S.""', 'Gross  Al""', 'Howe  John Wayne""',
       'Janos  James G. "Jesse Ventura" / McKinney  Cynthia""',
       'Jorgensen  Jo / Cohen  Jeremy "Spike"""',
       'Pierce  Brock / Ballard  Karla""', 'Sullivan  Dan""',
       'Trump  Donald J. / Pence  Michael R.""', 'Young  Don""'],
      dtype='object', name='Choice')


In [67]:
column_changes_dict = {'Biden  Joseph R. Jr. / Harris  Kamala D.""':"G20PREDBID",
                       'Jorgensen  Jo / Cohen  Jeremy "Spike"""':"G20PRELJOR",
                       'Trump  Donald J. / Pence  Michael R.""':"G20PRERTRU",
                       'Blankenship  Don / Mohr  William""':"G20PRECBLA",
       'De La Fuente  Rocque "Rocky" / Richardson  Darcy G.""':"G20PREOFUE",
        'Janos  James G. "Jesse Ventura" / McKinney  Cynthia""':"G20PREGJAN",
                       'Pierce  Brock / Ballard  Karla""':"G20PREIPIE", 
        'Gross  Al""':"G20USSDGRO", 
        'Howe  John Wayne""':"G20USSOHOW",
         'Sullivan  Dan""':"G20USSRSUL",
                       
       'Galvin  Alyse S.""':"G20HALDGAL",               
        'Young  Don""':"G20HALRYOU"
    
}

ak_2020_pvt = ak_2020_pvt.rename(columns=column_changes_dict)

In [72]:
print(ak_2020_pvt.head())

ak_2020_pvt = ak_2020_pvt[["Precinct",'G20PRERTRU', 'G20PREDBID', 'G20PRELJOR',
       'G20PREGJAN', 'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL',
       'G20USSDGRO', 'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']]

Choice                 Precinct  G20PREDBID  G20PRECBLA  G20PREOFUE  \
0                01-446 Aurora          239           3           0   
1       01-455 Fairbanks No. 1           47           0           0   
2       01-465 Fairbanks No. 2           85           1           1   
3       01-470 Fairbanks No. 3          128           0           1   
4       01-475 Fairbanks No. 4          103           2           0   

Choice  G20HALDGAL  G20USSDGRO  G20USSOHOW  G20PREGJAN  G20PRELJOR  \
0              293         246          46           2          43   
1               50          47           8           1           2   
2               98          79          23           1          12   
3              141         128          13           2          11   
4              110          91          16           1           5   

Choice  G20PREIPIE  G20USSRSUL  G20PRERTRU  G20HALRYOU  
0                3         465         466         452  
1                0          83        

In [73]:
print(vest_ak_20.columns)

Index(['DISTRICT', 'NAME', 'G20PRERTRU', 'G20PREDBID', 'G20PRELJOR',
       'G20PREGJAN', 'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL',
       'G20USSDGRO', 'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL', 'geometry'],
      dtype='object')


In [77]:
data_columns = ['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR',
       'G20PREGJAN', 'G20PRECBLA', 'G20PREIPIE', 'G20PREOFUE', 'G20USSRSUL',
       'G20USSDGRO', 'G20USSOHOW', 'G20HALRYOU', 'G20HALDGAL']

In [78]:
print("Printing differences below:")
for race in data_columns:
    if (sum(vest_ak_20[race])-sum(ak_2020_pvt[race]) != 0):
        print(race+" has a difference of "+str(sum(vest_ak_16[race])-sum(ak_2020_pvt[race]))+" votes")
        print("\tVEST: "+str(sum(vest_ak_20[race]))+" votes")
        print("\tSOURCES: "+str(sum(ak_2020_pvt[race]))+" votes")
print("")
print("All other races are equal")

Printing differences below:

All other races are equal


Early, Absentee, and Questioned votes are only reported at the State House district level (since Alaska has portions of the state with no lower level of government, HDs serve a similar purpose as counties do in other states for the purpose of reporting votes). These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within an HD. Similarly, federal-only ballots that are reported at the statewide level were apportioned to precincts by candidate based on their share of the precinct-level vote.